In [1]:
AGENT_NAME = 'default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SURROGATE = 'surrogates/imitator.pth'

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.estimators.classification import PyTorchClassifier as classifier

from art.attacks.evasion import FastGradientMethod as FGM

import logging
import pandas as pd
import numpy as np
import torch
from torch.nn import CrossEntropyLoss

import KBMproject.utilities as utils

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
agent = PPO.load(path=f"{AGENT_NAME}")
print('Model loaded from storage')

Model loaded from storage


In [5]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

In [6]:
import os
path = os.path.join(os.getcwd(), '..','observation_masks.csv')
path = os.path.normpath(path) #resolve '..'
observation_masks = pd.read_csv(path)

In [7]:
imitator = torch.load(SURROGATE)

In [13]:
kwargs = dict(norm=np.inf,
        eps=0.2,
        #num_random_init=1,
        #targeted=False,
        #minimal=True
        )
attack = FGM(estimator=classifier(
                        model=imitator, #[:-2],
                        loss=CrossEntropyLoss(), 
                        #nb_classes=agent.action_space[0].n, 
                        nb_classes=imitator[-1].out_features,
                        input_shape=agent.observation_space.shape,
                        device_type='gpu',
                        clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max())
                        ),
            **kwargs
             )

In [14]:
logging.basicConfig(level=logging.WARN)
kpi, obs, adv_obs, asr = utils.eval_untargeted_attack(agent, env, attack,
                                                      mask=observation_masks['mask_time'].to_numpy().astype(float),
                                                      time_steps=None)

100%|██████████| 8759/8759 [01:24<00:00, 103.91it/s, ASR=0.803]


In [15]:
display(kpi)

cost_function
annual_peak_average                      1.314043
carbon_emissions_total                   0.944874
cost_total                               0.874296
daily_one_minus_load_factor_average      1.042169
daily_peak_average                       1.096641
electricity_consumption_total            0.949741
monthly_one_minus_load_factor_average    1.013470
ramping_average                          1.508967
zero_net_energy                          1.114755
Name: District, dtype: float64

In [10]:
display(kpi)

cost_function
annual_peak_average                      1.034373
carbon_emissions_total                   0.923742
cost_total                               0.840175
daily_one_minus_load_factor_average      1.021646
daily_peak_average                       1.058670
electricity_consumption_total            0.933893
monthly_one_minus_load_factor_average    1.010315
ramping_average                          1.402107
zero_net_energy                          1.109749
Name: District, dtype: float64